In [1]:
include("../basis.jl")
include("../identity.jl")
include("../basic_gates.jl")
using Compat.Test
using BenchmarkTools

In [77]:
function xgate(num_bit::Int, bits::Ints)
    norder = flip(basis(num_bit), bits)
    PermuteMultiply(norder+1, ones(1<<num_bit))
end

function ygate(num_bit::Int, bits::Ints)
    norder = flip(basis(num_bit), bits)
    #vals = [-im*(-1)^reduce(+,takebit(b, bits)) for b in basis]
    #vals = mapreduce(bit->map(x->x==0?-im:im, takebit(basis, bit)), .*, bits)
    vals = mapreduce(bit->im.*(2.*takebit.(basis(num_bit), bit).-1), .*, bits)
    PermuteMultiply(norder+1, vals)
end

function zgate(num_bit::Int, bits::Ints)
    #vals = [(-1)^reduce(+,takebit(b, bits)) for b in basis]
    vals = mapreduce(bit->1.-2.*takebit.(basis(num_bit), bit), (x,y)->broadcast(*,x,y), bits)
    Diagonal(vals)
end

zgate (generic function with 1 method)

In [78]:
############################ TODO ################################
# arbitrary off-diagonal single qubit gate
# e.g. X, Y
function ndiaggate(num_bit::Int, gate::PermuteMultiply, bits::Ints)
    norder = flip(basis(num_bit), bits)
    vals = mapreduce(bit->exp.(im*phi*(2*takebit.(basis(num_bit), bit).-1)), .*, bits)
    PermuteMultiply(norder+1, vals)
end

# arbitrary diagonal single qubit gate
# e.g. Z, Rz(θ)
function diaggate(num_bit::Int, gate::Diagonal, bits::Ints)
    vals = mapreduce(bit->exp.(im*phi*(2*takebit.(basis(num_bit), bit).-1)), .*, bits)
    PermuteMultiply(basis(num_bit)+1, vals) # or Diagonal(vals) ?
end

#TODO?
# arbituary dense single qubit gate: SparseMatrixCSC
# e.g. Rx(θ), Ry(θ), Rot(θ1,θ2,θ3)
#function densegate(bits::Ints, basis::Vector{DInt})
#end

# arbituary control PermuteMultiply gate: SparseMatrixCSC
# swap gate
function swapgate(basis::Vector{DInt}, b1::Int, b2::Int)
end

swapgate (generic function with 1 method)

In [79]:
⊗ = kron
function superkron(num_bit::Int, gates::Vector{T}, locs::Vector{Int}) where T<:AbstractMatrix
    locs = num_bit - locs + 1
    order = sortperm(locs)
    _wrap_identity(gates[order], diff(vcat([0], locs[order], [num_bit+1])) - 1)
end

# kron, and wrap matrices with identities.
function _wrap_identity(data_list::Vector{T}, num_bit_list::Vector{Int}) where T<:AbstractMatrix
    length(num_bit_list) == length(data_list) + 1 || throw(ArgumentError())
    reduce((x,y)->x ⊗ y[1] ⊗ II(1<<y[2]), II(1 << num_bit_list[1]), zip(data_list, num_bit_list[2:end]))
end

_wrap_identity (generic function with 1 method)

In [80]:
@test superkron(4, [PAULI_X, PAULI_Y], [3,2]) == II(2) ⊗ PAULI_X ⊗ PAULI_Y ⊗ II(2)

Test Passed


In [81]:
# a function projecting space into target space.
function projector(basis::Vector{DInt}, cbits::Int)
    # Assigned: LW
end

projector (generic function with 1 method)

In [82]:
#@test projector(basis(6), [2, -3, -5]) == superkron(6, [P0, P1, P1], [2, 3, 5])

In [83]:
general_controlled_gates(num_bit::Int, projectors::Vector{Tp}, cbits::Vector{Int}, gates::Vector{Tg}, locs::Vector{Int}) where {Tg<:AbstractMatrix, Tp<:AbstractMatrix} = II(1<<num_bit) - superkron(num_bit, projectors, cbits) + superkron(num_bit, vcat(projectors, gates), vcat(cbits, locs))

general_controlled_gates (generic function with 1 method)

In [84]:
CNOT = PermuteMultiply(Complex128[1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0])
TOFFOLI = kron(P0, II(4)) + kron(P1, CNOT)
@test general_controlled_gates(2, [P1], [2], [PAULI_X], [1]) == CNOT

Test Passed


In [85]:
@test zgate(4, [1,2,3]) == superkron(4, [PAULI_Z, PAULI_Z, PAULI_Z], [1,2,3])

Test Passed


In [86]:
# Assigned: LW
# control gate for single qubit gate
function controlled_U1(num_bit::Int, gate::PermuteMultiply, b1::Int, b2::Int)
end
function controlled_U1(num_bit::Int, gate::SparseMatrixCSC, b1::Int, b2::Int)
end
function controlled_U1(num_bit::Int, gate::Diagonal{T}, b1::Int, b2::Int) where {T}
    vals = Vector{T}
    for b in basis(num_bit)
        println(b, b1)
        println(testbit(b, b1))
        if testbit(b, b1)
            println(test[b2])
            push!(vals, gate[takebit[b, b2]])
        else
            push!(vals, convert(T, 1.0))
        end
    end
    Diagonal(vals)
end
function controlled_U1(num_bit::Int, gate::StridedMatrix, b1::Int, b2::Int)
end

controlled_U1 (generic function with 8 methods)

In [87]:
# how to use push, and why not use push.
arr1 = zeros(3)
display(@benchmark push!([0,0,0],4))
# but, this is not recommended, allocate before setting elements!
arr2 = zeros(4)
display(@benchmark arr[4] = 4)

BenchmarkTools.Trial: 
  memory estimate:  176 bytes
  allocs estimate:  2
  --------------
  minimum time:     47.056 ns (0.00% GC)
  median time:      48.315 ns (0.00% GC)
  mean time:        61.275 ns (13.56% GC)
  maximum time:     1.929 μs (93.15% GC)
  --------------
  samples:          10000
  evals/sample:     989

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     22.105 ns (0.00% GC)
  median time:      34.649 ns (0.00% GC)
  mean time:        34.914 ns (0.00% GC)
  maximum time:     272.723 ns (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     996

In [88]:
println(controlled_U1(2, PAULI_Z, 1, 2))

01
false


LoadError: [91mMethodError: no method matching push!(::Type{Array{Complex{Int64},1}}, ::Complex{Int64})[0m
Closest candidates are:
  push!(::Any, ::Any, [91m::Any[39m) at abstractarray.jl:1935
  push!(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at abstractarray.jl:1936
  push!([91m::Array{Any,1}[39m, ::ANY) at array.jl:624
  ...[39m

In [89]:
# CNOT/CZ may be further accelerated.
function czgate(num_bit::Int, b1::Int, b2::Int)
    Diagonal(map(i->1-2*(takebit(i, b1) & takebit(i, b2)), basis(num_bit)))
end

czgate (generic function with 1 method)

In [74]:
@test czgate(2, 1, 2) == [1 0 0 0; 0 1 0 0; 0 0 1 0; 0 0 0 -1]
display(@benchmark czgate(16, 3, 7))
display(@benchmark general_controlled_gates(16, [P1], [7], [PAULI_Z], [3]))

BenchmarkTools.Trial: 
  memory estimate:  512.09 KiB
  allocs estimate:  3
  --------------
  minimum time:     64.454 μs (0.00% GC)
  median time:      81.313 μs (0.00% GC)
  mean time:        93.450 μs (12.07% GC)
  maximum time:     918.659 μs (85.37% GC)
  --------------
  samples:          10000
  evals/sample:     1

BenchmarkTools.Trial: 
  memory estimate:  9.28 MiB
  allocs estimate:  117
  --------------
  minimum time:     2.711 ms (0.00% GC)
  median time:      2.927 ms (0.00% GC)
  mean time:        3.199 ms (9.54% GC)
  maximum time:     5.909 ms (12.97% GC)
  --------------
  samples:          1557
  evals/sample:     1

In [90]:
@test general_controlled_gates(16, [P1], [7], [PAULI_Z], [3]) == czgate(16, 7, 3)

Test Passed


In [98]:
############# CNOT Gate as an example ##################
function cnotgate(num_bit::Int, b1::Int, b2::Int)
    mask = bmask(b1)
    db = b2-b1
    order = map(i->i ⊻ ((i .& mask) << db) + 1, basis(num_bit))
    PermuteMultiply(order, ones(Int, 1<<num_bit))
end

cnotgate (generic function with 1 method)

In [99]:
@test cnotgate(2, 2, 1) == [1 0 0 0; 0 1 0 0; 0 0 0 1; 0 0 1 0]
@test general_controlled_gates(16, [P1], [7], [PAULI_X], [3]) == cnotgate(16, 7, 3)

Test Passed


In [100]:
############# uncomment to run benchmarks ##################
@benchmark cnotgate(16, 7, 3)
#@benchmark general_controlled_gates(16, [P1], [7], [PAULI_X], [3])

BenchmarkTools.Trial: 
  memory estimate:  1.00 MiB
  allocs estimate:  6
  --------------
  minimum time:     151.186 μs (0.00% GC)
  median time:      165.721 μs (0.00% GC)
  mean time:        198.737 μs (13.33% GC)
  maximum time:     1.463 ms (80.96% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [94]:
toffoligate(basis::Vector{DInt}, b1::Int, b2::Int, b3::Int) = multicontrolled_U1(basis, PAULI_X, [b1, b2], b3)

toffoligate (generic function with 1 method)

In [95]:
# Assigned: LW
# multi-control gate for single qubit gate
function multicontrolled_U1(basis::Vector{DInt}, gate::PermuteMultiply, cbits::Vector{Int}, b2::Int)
end
# repeat for other matrix formats ...

multicontrolled_U1 (generic function with 1 method)

In [29]:
# TODO
# future, arbituary two site gate support
# probabily focus is enough? or, use projection decomposition?

In [12]:
#@test toffligate(3, 2, 3, 1) == TOFFOLI

8×8 SparseMatrixCSC{Complex{Float64},Int64} with 8 stored entries:
  [1, 1]  =  1.0+0.0im
  [2, 2]  =  1.0+0.0im
  [3, 3]  =  1.0+0.0im
  [4, 4]  =  1.0+0.0im
  [5, 5]  =  1.0+0.0im
  [6, 6]  =  1.0+0.0im
  [8, 7]  =  1.0+0.0im
  [7, 8]  =  1.0+0.0im

In [175]:
x = []
push!(x, Complex64(1.0))

1-element Array{Any,1}:
 1.0+0.0im